In [1]:
!pip install pandas 
!pip install sklearn

Unable to create process using 'C:\Users\Ethan Ohman\Anaconda3\python.exe "C:\Users\Ethan Ohman\Anaconda3\Scripts\pip-script.py" install pandas '
Unable to create process using 'C:\Users\Ethan Ohman\Anaconda3\python.exe "C:\Users\Ethan Ohman\Anaconda3\Scripts\pip-script.py" install sklearn'


In [2]:
import pandas as pd
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:
ticker_list = ['AAPL','ADBE','AMZN','BA','CMCSA','CSCO','CVX','DIS','FB','GOOGL','INTC','KO','LOW','MCD','MSFT','MU',
               'NFLX','NKE','PEP','SBUX','T','TGT','VZ','WMT','XOM']
bank_list = ['MORGAN STANLEY','EVERCORE ISI','JEFFERIES','JPMORGAN','OPPENHEIMER AND CO']

def switcher(month):
    cases = {
        'Jan':'01',
        'Feb':'02',
        'Mar':'03',
        'Apr':'04',
        'May':'05',
        'Jun':'06',
        'Jul':'07',
        'Aug':'08',
        'Sep':'09',
        'Oct':'10',
        'Nov':'11',
        'Dec':'12'
    }
    return cases.get(month,1)

def date_convert_target(date):
    date_list=date.split('-')
    day=date_list[0].strip()
    month=switcher(date_list[1].strip())
    year=date_list[2][2:]
#     print(len(date_list))
#     print("{0} {1} {2}yet".format(day, month, year))
    date_string = '{0}/{1}/{2}'.format(day, month, year)
    date_actual = datetime.datetime.strptime(date_string,'%d/%m/%y')
    return date_actual

def date_convert_price(date):
    
    date_list=date.split('/')
    date = "{0}/{1}/{2}".format(date_list[0],date_list[1],date_list[2][2:])
    date = datetime.datetime.strptime(date, "%m/%d/%y")
    return date

def re_order(data_frame, tickers):
    new_frame=pd.DataFrame()
    for ticker in tickers:
        dum_frame = data_frame[(data_frame['Ticker'] == ticker)]
        dum_frame = dum_frame.iloc[::-1]
        new_frame = new_frame.append(dum_frame)       
    return new_frame

def rename(stock):
    if(stock == 'APPL'):
        return 'AAPL'
    else:
        return stock 

In [4]:
#Cleaning Target Data
target_data = pd.read_csv('/Users/Ethan Ohman/Desktop/QMI Projects/Equity Research/main_data.csv')
target_data['Date'].value_counts()

## UNCOMMENTED FOR DEMO -- because main_data is actually the cleaned verison of original data

target_data['Date'] = target_data['Date'].apply(date_convert_target)
# # target_data['Ticker'] = target_data['Ticker'].apply(rename)
# # target_data=re_order(target_data,ticker_list)
# # target_data.index = [num for num in range(len(target_data))]
target_data.drop(['Unnamed: 0'],axis=1,inplace=True)
target_data.rename(columns={"Stock":"Ticker"}, inplace=True)
target_data.head()
# print(target_data.columns)
# target_data.shape

,Bank,Date,Position,Text,Price Target,Ticker
0,EVERCORE ISI,2019-11-21,outperform,"Technology | IT Hardware November 21, 2019 Am...",275.0,AAPL
1,MORGAN STANLEY,2019-11-19,dne,"November 19, 2019 04:17 AM GMT MORGAN STANLEY...",DNE,AAPL
2,EVERCORE ISI,2019-11-17,outperform,"Technology | IT Hardware November 17, 2019 Am...",275.0,AAPL
3,EVERCORE ISI,2019-11-16,dne,"Technology | IT Hardware November 16, 2019 Am...",DNE,AAPL
4,JPMORGAN,2019-11-15,overweight,North America Equity Research 15 November 201...,290.0,AAPL


In [5]:
#Cleaning Price Data
price_data = pd.read_csv('/Users/Ethan Ohman/Desktop/QMI Projects/Equity Research/pricedata.csv')
price_data.dropna(inplace=True)
drop_loc = price_data.loc[(price_data['Date'] == 'Date')].index[0]
price_data.drop([drop_loc],inplace=True)
price_data['Date'].value_counts()
price_data['Date'] = price_data['Date'].apply(date_convert_price)
price_data.drop(columns=['Unnamed: 0'],inplace=True)
price_data.head()
# day of, day after, 1 week after, 1 month after

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2014-12-30,AAPL,113.639999,113.919998,112.110001,112.519997,103.385315,29881500
1,2014-12-31,AAPL,112.82,113.129997,110.209999,110.379997,101.41906,41403400
2,2015-01-02,AAPL,111.389999,111.440002,107.349998,109.330002,100.4543,53204600
3,2015-01-05,AAPL,108.290001,108.650002,105.410004,106.25,97.624336,64285500
4,2015-01-06,AAPL,106.540001,107.43,104.629997,106.260002,97.633545,65797100


In [10]:
new_target = pd.DataFrame()
def get_close_prices(price_data, target_data, new_target, ticker_list, delta):
    interval=datetime.timedelta(delta)
    main_frame = []
    temp_frame = []
    for ticker in ticker_list:
        repeat = False
        min_date = price_data['Date'].min()
        dum_frame = target_data[(target_data['Ticker']==ticker)&(target_data['Date']>=min_date)]
        new_target = new_target.append(dum_frame)
        price_dum_frame = price_data[(price_data['Ticker']==ticker)]
        max_date = price_dum_frame['Date'].max()
        max_value = price_dum_frame.loc[price_dum_frame['Date']==max_date, 'Close']
        max_value = float(max_value)
        index = 0 
        for date in dum_frame['Date']:
            length = len(dum_frame['Date'])
            while(date+interval > price_dum_frame.iloc[index]['Date']):
                index+=1 
                if(index >= len(price_dum_frame['Date'])):
                    temp_frame.append(max_value)
                    repeat = True
                    break    
            if(repeat):
                break
            if(index<len(price_dum_frame['Date'])):
                if(date+interval<=price_dum_frame.iloc[index]['Date']):
                    temp_frame.append(price_dum_frame.iloc[index]['Close'])
#         print(ticker)
#         print(len(temp_frame))
        temp_frame += [max_value]*(length-len(temp_frame))
        main_frame += temp_frame
        temp_frame.clear()
    return (main_frame,new_target)
day_of = get_close_prices(price_data, target_data, new_target, ticker_list, 0)
one_day = get_close_prices(price_data, target_data, new_target, ticker_list, 1)
one_week = get_close_prices(price_data, target_data, new_target, ticker_list, 7)
one_month = get_close_prices(price_data, target_data, new_target, ticker_list, 30)

In [11]:
# DEMO - DO NOT PRESS THIS

new_target = day_of[1]
print(len(day_of[0]))
print(len(one_day[0]))
print(len(one_week[0]))
print(len(one_month[0]))

tickers = pd.get_dummies(new_target['Ticker'])
new_target = pd.concat([tickers,new_target],axis=1)
print(new_target.shape)
dates = new_target['Date']
new_target.drop(columns=['Ticker','Date'],inplace = True)
# new_target['Mean Tgt Price'] = [lambda x: float(x) for x in new_target['Mean Tgt Price']]
# new_target.drop(['Mean Tgt Price'], axis=1)
new_target = pd.concat([dates,new_target],axis=1)

new_target['0 days'] = day_of[0]
new_target['1 days'] = one_day[0]
new_target['7 days'] = one_week[0]
new_target['30 days'] = one_month[0]
# new_target.head()
# new_target.to_csv('/Users/Ethan Ohman/Desktop/target_data_part1.csv')



6787
6787
6787
6787
(6787, 31)


In [12]:
new_target.head()

,Date,AAPL,ADBE,AMZN,BA,CMCSA,CSCO,CVX,DIS,FB,...,WMT,XOM,Bank,Position,Text,Price Target,0 days,1 days,7 days,30 days
0,2019-11-21,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,outperform,"Technology | IT Hardware November 21, 2019 Am...",275.0,262.01001,261.779999,267.25,264.16
1,2019-11-19,1,0,0,0,0,0,0,0,0,...,0,0,MORGAN STANLEY,dne,"November 19, 2019 04:17 AM GMT MORGAN STANLEY...",DNE,262.01001,261.779999,267.25,264.16
2,2019-11-17,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,outperform,"Technology | IT Hardware November 17, 2019 Am...",275.0,262.01001,261.779999,267.25,264.16
3,2019-11-16,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,dne,"Technology | IT Hardware November 16, 2019 Am...",DNE,262.01001,261.779999,267.25,264.16
4,2019-11-15,1,0,0,0,0,0,0,0,0,...,0,0,JPMORGAN,overweight,North America Equity Research 15 November 201...,290.0,262.01001,261.779999,267.25,264.16


In [13]:
## Modeling
data = pd.read_csv('/Users/Ethan Ohman/Desktop/QMI Projects/Equity Research/target_data_part1.csv')
data.drop(['Mean Tgt Price', 'Mean Description', 'Mean Rating'], axis=1, inplace=True)

X = data.iloc[:,2:35]
full_y = data.iloc[:,35:39]

complete_model_dict = {'0 days':[None,[]], '1 days':[None,[]], '7 days':[None,[]], '30 days':[None,[]]}





In [14]:
for timeframe in full_y:
    print((timeframe))
    y = full_y[timeframe]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)
    regressor = LinearRegression()    
    regressor.fit(X_train,y_train)
    for idx, col_name in enumerate(X_train.columns):
        strr = "{0} coefficient: {1}".format(col_name, regressor.coef_[idx])
        complete_model_dict[timeframe][1].append(strr)
        score = regressor.score(X_test,y_test)
        complete_model_dict[timeframe][0] = score

0 days
1 days
7 days
30 days


In [15]:
#Visualize Models (Basic stuff)
for title, info in complete_model_dict.items():
    print(title.upper())
    print("R^2 Score: {0}".format(info[0]))
    print("Individual Coeffs:")
    for coeff in info[1]:
        print(coeff)
    print("\n")

0 DAYS
R^2 Score: 0.7632891890623307
Individual Coeffs:
AAPL coefficient: -9.372270433328787
ADBE coefficient: -44.026769871202355
AMZN coefficient: 866.0276332005793
BA coefficient: 112.94190549039153
CMCSA coefficient: -187.84820072877983
CSCO coefficient: -93.72579342577518
CVX coefficient: -71.33470638981719
DIS coefficient: -77.05948918946027
FB coefficient: -133.6566981620545
GOOGL coefficient: 614.7552858212774
INTC coefficient: -97.65612012273725
KO coefficient: -85.87728578993072
LOW coefficient: -95.9265763437981
MCD coefficient: 46.466375377098814
MSFT coefficient: -181.9368802871855
MU coefficient: -146.8993736972873
NFLX coefficient: -6.209787401554938
NKE coefficient: -115.11154745351382
PEP coefficient: -18.386523642596128
SBUX coefficient: -110.34162645918845
T coefficient: -48.25266322038217
TGT coefficient: -27.14764389459078
VZ coefficient: -18.73153999242213
WMT coefficient: -32.86420053267928
XOM coefficient: -37.82550285107374
# of Ratings coefficient: -26.3339773

In [16]:
new_target.head(20)

,Date,AAPL,ADBE,AMZN,BA,CMCSA,CSCO,CVX,DIS,FB,...,WMT,XOM,Bank,Position,Text,Price Target,0 days,1 days,7 days,30 days
0,2019-11-21,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,outperform,"Technology | IT Hardware November 21, 2019 Am...",275.0,262.01001,261.779999,267.25,264.16
1,2019-11-19,1,0,0,0,0,0,0,0,0,...,0,0,MORGAN STANLEY,dne,"November 19, 2019 04:17 AM GMT MORGAN STANLEY...",DNE,262.01001,261.779999,267.25,264.16
2,2019-11-17,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,outperform,"Technology | IT Hardware November 17, 2019 Am...",275.0,262.01001,261.779999,267.25,264.16
3,2019-11-16,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,dne,"Technology | IT Hardware November 16, 2019 Am...",DNE,262.01001,261.779999,267.25,264.16
4,2019-11-15,1,0,0,0,0,0,0,0,0,...,0,0,JPMORGAN,overweight,North America Equity Research 15 November 201...,290.0,262.01001,261.779999,267.25,264.16
5,2019-11-06,1,0,0,0,0,0,0,0,0,...,0,0,JPMORGAN,overweight,North America Equity Research 06 November 201...,280.0,262.01001,261.779999,267.25,264.16
6,2019-11-04,1,0,0,0,0,0,0,0,0,...,0,0,EVERCORE ISI,outperform,"Technology | IT Hardware November 04, 2019 Am...",262.0,262.01001,261.779999,267.25,264.16
7,2019-11-01,1,0,0,0,0,0,0,0,0,...,0,0,JEFFERIES,dne,EQUITY RESEARCH Global | Equity Strategy Stra...,DNE,262.01001,261.779999,267.25,264.16
8,2019-11-01,1,0,0,0,0,0,0,0,0,...,0,0,MORGAN STANLEY,dne,"2019 å¹´ 11 æœˆ 1 æ—¥ 06:52 AM GMT Apple, Inc...",DNE,262.01001,261.779999,267.25,264.16
9,2019-10-31,1,0,0,0,0,0,0,0,0,...,0,0,MORGAN STANLEY,overweight,"October 31, 2019 01:11 AM GMT MORGAN STANLEY ...",296.0,262.01001,261.779999,267.25,264.16


In [17]:
## NEW METRIC SECTION:

##THIS JUST SO I CAN GET A FRESH ENVIORNMENT
main_data = pd.read_csv('/Users/Ethan Ohman/Desktop/QMI Projects/Equity Research/main_data.csv')
main_data['Date'].value_counts()

## UNCOMMENTED FOR DEMO -- because main_data is actually the cleaned verison of original data

main_data['Date'] = main_data['Date'].apply(date_convert_target)
# # target_data['Ticker'] = target_data['Ticker'].apply(rename)
# # target_data=re_order(target_data,ticker_list)
# # target_data.index = [num for num in range(len(target_data))]
main_data.drop(['Unnamed: 0'],axis=1,inplace=True)
main_data.rename(columns={"Stock":"Ticker"}, inplace=True)
main_data.head()

,Bank,Date,Position,Text,Price Target,Ticker
0,EVERCORE ISI,2019-11-21,outperform,"Technology | IT Hardware November 21, 2019 Am...",275.0,AAPL
1,MORGAN STANLEY,2019-11-19,dne,"November 19, 2019 04:17 AM GMT MORGAN STANLEY...",DNE,AAPL
2,EVERCORE ISI,2019-11-17,outperform,"Technology | IT Hardware November 17, 2019 Am...",275.0,AAPL
3,EVERCORE ISI,2019-11-16,dne,"Technology | IT Hardware November 16, 2019 Am...",DNE,AAPL
4,JPMORGAN,2019-11-15,overweight,North America Equity Research 15 November 201...,290.0,AAPL


In [18]:
#THINK OF A DF being a NUMPY Matrix -- (at least when it comes to indexing)
def re_order(data_frame, tickers):
    new_frame=pd.DataFrame()
    for ticker in tickers:
        dum_frame = data_frame[(data_frame['Ticker'] == ticker)]
        dum_frame = dum_frame.iloc[::-1]
        new_frame = new_frame.append(dum_frame)       
    return new_frame
    

In [32]:
#Cleaning Price Data
price_data = pd.read_csv('/Users/Ethan Ohman/Desktop/QMI Projects/Equity Research/pricedata.csv')
price_data.dropna(inplace=True)
drop_loc = price_data.loc[(price_data['Date'] == 'Date')].index[0]
price_data.drop([drop_loc],inplace=True)
price_data['Date'].value_counts()
price_data['Date'] = price_data['Date'].apply(date_convert_price)
price_data['Close'] = price_data['Close'].apply(lambda x: float(x))
price_data.drop(columns=['Unnamed: 0'],inplace=True)

f_price_data = re_order(price_data, tickers)
f_price_data.head()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
1239,2019-12-02,AAPL,267.269989,268.25,263.450012,264.160004,264.160004,23621800
1238,2019-11-29,AAPL,266.600006,268,265.899994,267.250000,267.25,11654400
1237,2019-11-27,AAPL,265.579987,267.980011,265.309998,267.839996,267.839996,16308900
1236,2019-11-26,AAPL,266.940002,267.160004,262.5,264.290009,264.290009,26301900
1235,2019-11-25,AAPL,262.709991,266.440002,262.519989,266.369995,266.369995,21005100


In [33]:
#check that both data_frames have date time objects
print(type(main_data['Date'][0]))
print(type(f_price_data['Date'][0]))

print(type(f_price_data['Close'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.float64'>


In [34]:
#sustained direction metric - returns an inorder list which matches the length of the original d_f

def sustained_direction(main_data, price_data, tickers):
    metric_arr = []
    for ticker in tickers:
        ticker_metric_arr = []
        
        sliced_ticker_frame = main_data[(main_data['Ticker'] == ticker)]
        sliced_price_frame = price_data[(price_data['Ticker'] == ticker)]
        
        #(max,min)_date is cutoff by price_data
        max_date = sliced_price_frame['Date'].max()
        min_date = sliced_price_frame['Date'].min()
        frame_length = len(sliced_ticker_frame.index)
        
        sliced_ticker_frame = sliced_ticker_frame[(sliced_ticker_frame['Date'] <= max_date)]
        front_length = len(sliced_ticker_frame.index)
        sliced_ticker_frame = sliced_ticker_frame[(sliced_ticker_frame['Date'] >= min_date)]
        back_length = front_length - len(sliced_ticker_frame.index)
        #use front and back length later to create dummy values to ensure entire list is proper length
        length = len(sliced_ticker_frame.index)
        
        price_p = 0
        ticker_p = 0
        for ticker_date in sliced_ticker_frame['Date']:
            # main_data and price_data are sorted
            while(ticker_date < sliced_price_frame.iloc[price_p]['Date']):
                price_p = price_p + 1
            if(price_p > 0):
                increasing = True if (sliced_price_frame.iloc[price_p-1]['Close'] - sliced_price_frame.iloc[price_p]['Close'] > 0) else False
                lag = increasing
                temp_price_p= price_p
                days = 0
                total = 0
                while(temp_price_p >= 1):
                    diff = sliced_price_frame.iloc[price_p-1]['Close'] - sliced_price_frame.iloc[price_p]['Close']
                    increasing = True if  diff > 0 else False
                    if(increasing == lag):
                        days = days + 1
                        total = total + diff
                    else:
                        break
                    temp_price_p = temp_price_p - 1
                ticker_metric_arr.append(total/days)
            else: 
                ticker_metric_arr.append('DNE') ## as the sustained direction can't be read
        front_arr = ['DNE' for x in range(front_length)]
        back_arr = ['DNE' for x in range(back_length)]
        total_arr = front_arr + ticker_metric_arr + back_arr
        metric_arr = metric_arr + total_arr
    return metric_arr
            # (while)iterate to the proper point where main_data date and price_date are the same
            # initial direction (bool true of false)
            # while(ind > -1 and direction maintained)
                #add diff to total
                #increment days
            #calc metric and append to metric array
        #init DNE arrays
        #total_arr = front_dne + middle + back_dne
        #append to metric_ar
        
    
    

In [35]:
test_arr = sustained_direction(main_data, f_price_data, ticker_list)
print(test_arr)

6
8
10
10
10
17
19
20
20
21
21
21
21
21
21
22
22
22
24
24
24
25
25
25
26
28
34
34
38
40
42
44
48
53
54
54
55
57
57
58
58
58
59
59
60
60
61
63
64
66
74
77
80
81
84
85
86
86
86
86
86
87
87
88
88
90
90
91
92
92
93
93
94
101
102
102
102
103
103
108
109
113
113
114
115
116
116
117
118
120
121
123
124
125
125
126
127
127
129
130
140
143
144
149
149
149
149
151
153
154
160
161
164
165
166
168
171
172
174
175
177
182
183
184
198
201
206
209
210
211
211
212
212
212
212
214
214
215
224
226
227
228
228
230
230
230
237
238
239
245
245
246
246
247
251
261
261
264
265
267
267
268
268
270
270
270
270
272
272
273
273
274
274
277
281
282
287
294
296
298
302
306
306
310
310
311
312
314
315
321
322
334
335
336
336
336
337
342
343
350
351
364
375
376
379
381
382
384
389
396
396
399
400
400
404
407
408
426
427
451
458
459
459
460
460
464
465
466
466
470
472
487
488
496
501
503
508
518
519
520
521
521
553
554
554
558
558
559
559
560
586
587
587
588
588
597
597
598
600
616
620
623
627
627
627
628
628
630
637

1083
1083
1083
1083
1083
1084
1085
1087
1092
1098
1098
1113
1113
1114
1128
1128
1128
1135
1136
1139
1146
1146
1146
1147
1147
1148
1150
1153
1154
1154
1176
1194
1199
1203
1209
1209
1209
1210
1210
1210
1213
1216
1219
5
6
6
19
19
20
20
20
20
32
44
83
83
84
84
84
84
107
132
141
143
148
152
152
152
152
158
159
161
161
166
184
188
189
189
191
194
209
209
210
210
210
210
212
226
243
247
248
251
269
269
270
270
270
279
284
290
299
303
338
338
339
339
339
344
366
401
402
402
402
402
402
406
410
411
424
429
434
439
439
439
441
442
459
459
460
460
460
460
462
468
498
498
498
501
502
525
525
525
526
776
776
777
777
784
801
827
840
840
841
841
841
843
859
877
903
903
904
904
904
906
920
923
940
940
941
942
966
966
966
967
967
968
1000
1006
1007
1017
1027
1027
1028
1028
1028
1028
1030
1049
1053
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
1058
105

1034
1034
1034
1035
1058
1098
1098
1098
1130
1130
1130
1130
1150
1150
1153
1153
1154
1154
1154
1161
1161
1162
1162
1162
1162
1174
1179
1180
1192
1193
1193
1193
1197
1212
1212
1212
1214
1219
1219
1219
1222
1223
1223
1223
1238
9
15
16
17
24
25
25
26
26
26
26
26
26
27
27
30
30
30
30
31
36
37
47
49
59
89
93
94
94
94
94
94
94
95
95
95
96
97
97
98
100
100
102
106
107
111
111
120
123
127
128
131
134
140
148
152
153
153
153
153
153
154
154
155
157
157
158
166
167
168
172
194
195
197
210
211
211
211
211
212
212
212
214
215
217
224
225
239
245
246
275
276
276
276
276
276
277
278
278
279
283
284
285
286
296
302
302
304
311
317
317
318
318
343
344
344
344
344
344
345
345
347
348
352
353
369
371
373
373
373
374
376
377
377
377
377
377
378
391
395
395
398
400
401
402
402
402
402
402
402
403
407
407
408
415
416
422
423
425
425
456
460
460
461
461
461
461
462
462
464
466
466
469
498
524
525
525
526
526
526
526
527
527
527
527
528
530
532
549
550
550
550
553
553
553
554
554
559
561
562
562
562
578
584


949
950
950
950
950
950
992
1013
1014
1015
1015
1015
1015
1015
1045
1058
1078
1079
1079
7
14
24
25
25
25
25
25
25
64
84
85
85
85
85
85
85
147
154
154
154
155
155
155
155
155
180
183
196
199
203
212
212
212
213
213
213
227
239
242
244
246
250
260
260
269
274
277
278
278
278
278
278
293
305
307
308
312
314
338
341
341
342
342
342
342
342
366
373
378
385
386
392
393
403
404
404
404
405
405
405
405
405
425
444
467
467
467
468
468
468
468
468
468
470
471
501
503
504
530
530
531
531
531
532
532
532
532
532
532
547
551
562
564
564
572
587
590
590
590
590
590
591
591
591
591
602
609
644
644
644
646
646
652
653
658
658
658
659
659
659
659
659
660
673
676
705
705
705
712
716
717
718
718
718
719
719
719
719
723
744
745
746
779
782
782
783
783
783
783
783
786
806
809
820
842
842
843
843
843
844
844
844
845
848
855
858
858
868
878
879
880
887
894
907
908
909
909
909
910
910
910
910
910
910
912
918
920
950
968
972
972
972
972
973
973
973
973
973
973
973
975
975
1014
1026
1026
1027
1035
1035
1035
103